<a href="https://colab.research.google.com/github/panimesh14/RAG-Powered-Conversational-AI-Agent-with-FAISS-and-Augmented-Memory/blob/main/Conversational_RAG_powered_AI_Agent_using_FAISS_and_Contextual_Memory_for_reading_PDFs_and_Financial_Reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
%pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 7.8 MB/s eta 0:00:00


In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [6]:
# 1. Import libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os

In [8]:
# 2. Load the PDF document
pdf_path = "Exxonmobil_Report.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()  # Extracts text from each page as separate Document objects

In [9]:
# 3. Split the PDF into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)  # Helps the retriever by chunking large documents

In [10]:
from langchain_community.vectorstores import FAISS
!pip install -qU langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.4 MB/s eta 0:00:00


In [11]:
# 4. Embed the text chunks using a sentence-transformer
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(docs, embedding_model)  # Creates vector index using FAISS


/tmp/ipython-input-11-4050901680.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mo

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from huggingface_hub import notebook_login # Import notebook_login

# Log in to Hugging Face - this will prompt you to enter your token
# Ensure you have accepted the model's terms and conditions on the HF website
try:
    notebook_login()
except Exception as e:
    print(f"Hugging Face login failed: {e}")
    print("Please make sure you have accepted the terms for the model on the Hugging Face website and have a valid token.")

In [14]:
# 5. Load the open-source LLM (e.g., Mistral 7B)
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [15]:
# 6. Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.1
)

Device set to use cuda:0


In [16]:
# 7. Wrap the pipeline for LangChain
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipython-input-16-3267213372.py:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [20]:
# 8. Add memory to maintain chat history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer" # Explicitly set the output key to save the 'answer'
)

In [21]:
# 9. Create Conversational Retrieval Chain (RAG pipeline)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever(),
    memory=memory,
    return_source_documents=True
)

In [22]:
# 10. Run a simple loop to chat with the PDF
print("📘 Ask questions about the PDF (type 'exit' to quit):")
chat_history = []

while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit"]:
        break

    result = qa_chain({"question": query})
    print("\n🤖 Bot:", result["answer"])

📘 Ask questions about the PDF (type 'exit' to quit):

You: Summarize the document in 100 words


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
115

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
86

2.610% notes due 2030 2,000 2,000
2.995% notes due 2039 750 750
4.227% notes due 2040 2,076 2,080
3.567% notes due 2045 1,000 1,000
4.114% notes due 2046 2,500 2,500
3.095% notes due 2049 1,500 1,500
4.327% notes due 2050 2,750 2,750
3.452% notes due 2051 2,750 2,750
Exxon MobilC orporation -E uro-denominated
0.524% notes due 2028 1,039 1,105
0.835% notes due 2032 1,039 1,105
1.408% notes due 2039 1,039 1,105
XTOE nergyI nc. (4)
6.100% senior notes due 2036 187 189

returns through disciplined  
capital allocation
See supplemental information for footnotes, definitions, and cautionary statement beginning 
on page 138 in this report.
143

Question: Summarize the document in 100 words
Helpful Answer: This note provides a summ

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

2.610% notes due 2030 2,000 2,000
2.995% notes due 2039 750 750
4.227% notes due 2040 2,076 2,080
3.567% notes due 2045 1,000 1,000
4.114% notes due 2046 2,500 2,500
3.095% notes due 2049 1,500 1,500
4.327% notes due 2050 2,750 2,750
3.452% notes due 2051 2,750 2,750
Exxon MobilC orporation -E uro-denominated
0.524% notes due 2028 1,039 1,105
0.835% notes due 2032 1,039 1,105
1.408% notes due 2039 1,039 1,105
XTOE nergyI nc. (4)
6.100% senior notes due 2036 187 189

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
86

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
115

6.100% senior notes due 2036 187 189
6.750% senior notes due 2037 284 286
6.375% senior notes due 2038 221 223
PioneerN atural Resources Company (5)
1.125% senior notes due 2026 718 —
5.100%s eniorn otes due 2026 1,097 —
7.200% senior notes due 2028 250 —
1.900%

...


🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

2.610% notes due 2030 2,000 2,000
2.995% notes due 2039 750 750
4.227% notes due 2040 2,076 2,080
3.567% notes due 2045 1,000 1,000
4.114% notes due 2046 2,500 2,500
3.095% notes due 2049 1,500 1,500
4.327% notes due 2050 2,750 2,750
3.452% notes due 2051 2,750 2,750
Exxon MobilC orporation -E uro-denominated
0.524% notes due 2028 1,039 1,105
0.835% notes due 2032 1,039 1,105
1.408% notes due 2039 1,039 1,105
XTOE nergyI nc. (4)
6.100% senior notes due 2036 187 189

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
86

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
115

6.100% senior notes due 2036 187 189
6.750% senior notes due 2037 284 286
6.375% senior notes due 2038 221 223
PioneerN atural Resources Company (5)
1.125% senior notes due 2026 718 —
5.100%s eniorn otes due 2026 1,097 —
7.200% senior notes due 2028 250 —
1.900%

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

2.610% notes due 2030 2,000 2,000
2.995% notes due 2039 750 750
4.227% notes due 2040 2,076 2,080
3.567% notes due 2045 1,000 1,000
4.114% notes due 2046 2,500 2,500
3.095% notes due 2049 1,500 1,500
4.327% notes due 2050 2,750 2,750
3.452% notes due 2051 2,750 2,750
Exxon MobilC orporation -E uro-denominated
0.524% notes due 2028 1,039 1,105
0.835% notes due 2032 1,039 1,105
1.408% notes due 2039 1,039 1,105
XTOE nergyI nc. (4)
6.100% senior notes due 2036 187 189

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
86

NOTES TO CONSOLIDATED FINANCIAL STATEMENTS
115

6.100% senior notes due 2036 187 189
6.750% senior notes due 2037 284 286
6.375% senior notes due 2038 221 223
PioneerN atural Resources Company (5)
1.125% senior notes due 2026 718 —
5.100%s eniorn otes due 2026 1,097 —
7.200% senior notes due 2028 250 —
1.900%

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 312.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 42.12 MiB is free. Process 9337 has 14.70 GiB memory in use. Of the allocated memory 13.79 GiB is allocated by PyTorch, and 796.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)